In [1]:
import os
import numpy as np
import pandas as pd
import psycopg2



In [2]:
df = pd.read_csv('Customer Contracts$.csv')
df.head()

,customer_name,start_date,end_date,contract_amount_m,invoice_sent,paid
0,Nike,01-02-2019,12-20-2020,2.98,Yes,Yes
1,Reebox,06-20-2017,NaN,3.90,No,No
2,Adidas,12-07-2015,6-20-2018,4.82,Yes,Yes
3,Google,05-25-2014,03-20-2017,5.74,Yes,No
4,Amazon,11-10-2012,12-20-2015,6.66,No,Yes


In [3]:
#clean our file name, all lower-case, remove white spaces and replace unecessary with fine 
raw_file = "Customer Contracts$"
raw_to_formatted_file = raw_file.lower().replace(" ", "_").replace("-","_").replace(r"/","_").replace("\\","_").replace("$","").replace("%","")
raw_to_formatted_file


'customer_contracts'

In [5]:
#clean our data columns, all lower-case, remove white spaces and replace unecessary with fine 
df.columns = [x.lower().replace(" ", "_").replace("-","_").replace(r"/","_").replace("\\","_").replace("$","").replace("%","") for x in df.columns]
df.columns

Index(['customer_name', 'start_date', 'end_date', 'contract_amount_m',
       'invoice_sent', 'paid'],
      dtype='object')

In [ ]:
create table customer_contracts
(
    customer_name           varchar,
    start_date              varchar,
    end_date                varchar,
    contract_amount_m       float,
    invoice_sent            varchar,
    paid                    varchar
);

In [6]:
df.dtypes

customer_name         object
start_date            object
end_date              object
contract_amount_m    float64
invoice_sent          object
paid                  object
dtype: object

In [7]:
 #processing data, changing the data type of raw data from object to a sql recognised data types
change_datatype = {
        'timedelta64[ns]': 'varchar',
        'object': 'varchar',
        'float64': 'float',
        'int64': 'int',
        'datetime64': 'timestamp'
    }
change_datatype

{'timedelta64[ns]': 'varchar',
 'object': 'varchar',
 'float64': 'float',
 'int64': 'int',
 'datetime64': 'timestamp'}

In [8]:
#replacing the columns and their respetcive data type from panda to sql
col_str = ", ".join("{} {}".format(n, d) for (n, d) in zip(df.columns, df.dtypes.replace(change_datatype)))
col_str

'customer_name varchar, start_date varchar, end_date varchar, contract_amount_m float, invoice_sent varchar, paid varchar'

In [11]:
#open database connection
conn_string = "host='localhost'dbname='bigdata'user='postgres'password='Reason@2010'"
conn = psycopg2.connect(conn_string)
cursor = conn.cursor()
print("DB Connection established successfully")

DB Connection established successfully


In [12]:
#drop tables with same name
cursor.execute("drop table if exists customer_contracts;")

In [13]:
#create a table
cursor.execute("create table customer_contracts (customer_name varchar, start_date varchar, end_date varchar, contract_amount_m float, invoice_sent varchar, paid varchar)")

In [14]:
#insert into the table and save df to csv, open csv and upload to db
df.to_csv('customer_contracts.csv', header=df.columns, index=False, encoding='utf-8')
my_file = open('customer_contracts.csv')
print("File open in memory")



File open in memory


In [15]:
#upload to db
SQL_STATEMENT = """
    COPY customer_contracts FROM STDIN WITH
        CSV
        HEADER
        DELIMITER AS ','
    """
cursor.copy_expert(sql=SQL_STATEMENT, file=my_file)
print("file uploaded to db")

file uploaded to db


In [16]:
#permission on db in case of multiple users optional
cursor.execute("grant select on customer_contracts to public")
conn.commit()

cursor.close()
print("data upload to db completed")

data upload to db completed
